In [11]:
import requests
import pandas as pd

In [12]:
OPENAQ_API_KEY = "2df1e1ad492dbcc9293e82253f0ddffca796f334d424d0c1765e827c87a219ea"

BASE_URL = "https://api.openaq.org/v3/"

In [13]:
headers = {
    "X-API-Key": OPENAQ_API_KEY
}

In [14]:
endpoint_parameters = f"{BASE_URL}parameters"
endpoint_locations = f"{BASE_URL}locations"

In [15]:
major_us_cities = [
    "Los Angeles", "New York", "Chicago", "Houston", "Phoenix",
    "Philadelphia", "San Antonio", "San Diego", "Dallas", "San Jose"
]


In [16]:
parameters_params = {
    "limit": 10,
}

get_parameters is used for getting the unit used for measuring respective data

In [17]:
def get_parameters():
    response = requests.get(endpoint_parameters, headers=headers, params=parameters_params)
    if response.status_code == 200:
        data = response.json()
        para_df = pd.DataFrame(data['results'])
        print("Parameters:")
        print(para_df)
        return para_df
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return pd.DataFrame()

In [18]:
def get_locations():
    all_cities_with_data = []

    for city in major_us_cities:
        params = {
            "country": "US", "limit": 5, "page": 1
            }
        response = requests.get(endpoint_locations, headers=headers, params=params)
        data = response.json()
    
        if "results" in data:
            city_data = data['results']
            if city_data:
                city_df = pd.DataFrame(city_data)
                city_df['city'] = city
                all_cities_with_data.append(city_df)
            else:
                print(f"No data found for {city}.")

        else:
            # print(f"Error: {response.status_code} - {response.text}")
            pass
    if all_cities_with_data:
        all_cities_df = pd.concat(all_cities_with_data, ignore_index=True)
        print("Locations:")
        print(all_cities_df)
    else:
        print("No locations found for the specified cities.")
        return pd.DataFrame()

In [19]:
def main():
    get_parameters()
    get_locations()

In [20]:
if __name__ == "__main__":
    main()

Parameters:
   id  name  units displayName  \
0   1  pm10  µg/m³        PM10   
1   2  pm25  µg/m³       PM2.5   
2   3    o3  µg/m³     O₃ mass   
3   4    co  µg/m³     CO mass   
4   5   no2  µg/m³    NO₂ mass   
5   6   so2  µg/m³    SO₂ mass   
6   7   no2    ppm         NO₂   
7   8    co    ppm          CO   
8   9   so2    ppm         SO₂   
9  10    o3    ppm          O₃   

                                         description  
0  Particulate matter less than 10 micrometers in...  
1  Particulate matter less than 2.5 micrometers i...  
2                           Ozone mass concentration  
3                 Carbon Monoxide mass concentration  
4                Nitrogen Dioxide mass concentration  
5                  Sulfur Dioxide mass concentration  
6                     Nitrogen Dioxide concentration  
7                      Carbon Monoxide concentration  
8                       Sulfur Dioxide concentration  
9                                Ozone concentration  
Location

Code not working. Likely due to the fact that JSON for locations uses locality, and not city.